Market depth (order book)
==============

In [21]:
from ib_async import *

util.startLoop()

ib = IB()
ib.connect("127.0.0.1", 4001, clientId=16)

<IB connected to 127.0.0.1:4001 clientId=16>

Error 1100, reqId -1: Connectivity between IB and Trader Workstation has been lost.
Error 1100, reqId -1: Connectivity between IB and Trader Workstation has been lost.
Error 317, reqId 6: Market depth data has been RESET. Please empty deep book contents before applying any new entries., contract: Contract(secType='FUT', conId=620730920, symbol='NQ', lastTradeDateOrContractMonth='20240621', multiplier='20', exchange='CME', currency='USD', localSymbol='NQM4', tradingClass='NQ')


In [ ]:
from IPython.display import display, clear_output
import pandas as pd

df = pd.DataFrame(index=range(5), columns="bidSize bidPrice askPrice askSize".split())

# NQM2024 contract
contract = Contract(conId=620730920)

ib.qualifyContracts(contract)
ticker = ib.reqMktDepth(contract)


def onTickerUpdate(ticker):
    bids = ticker.domBids
    for i in range(5):
        df.iloc[i, 0] = bids[i].size if i < len(bids) else 0
        df.iloc[i, 1] = bids[i].price if i < len(bids) else 0
    asks = ticker.domAsks
    for i in range(5):
        df.iloc[i, 2] = asks[i].price if i < len(asks) else 0
        df.iloc[i, 3] = asks[i].size if i < len(asks) else 0
    clear_output(wait=True)
    display(df)


ticker.updateEvent += onTickerUpdate

IB.sleep(15)

,bidSize,bidPrice,askPrice,askSize
0,0,0,0,0
1,0,0,0,0
2,0,0,0,0
3,0,0,0,0
4,0,0,0,0


In [14]:
ib.cancelMktDepth(contract)

In [16]:
ib.positions(account="U10394496")

[Position(account='U10394496', contract=Future(conId=620730920, symbol='NQ', lastTradeDateOrContractMonth='20240621', multiplier='20', currency='USD', localSymbol='NQM4', tradingClass='NQ'), position=6.0, avgCost=344900.6149234)]

In [17]:
ib.orders()

[Order(permId=827631043, action='BUY', totalQuantity=1.0, orderType='LMT', lmtPrice=17248.0, auxPrice=0.0, tif='GTC', ocaType=3, orderRef='cLnpF0mGeHCl', displaySize=2147483647, rule80A='0', openClose='', volatilityType=0, deltaNeutralOrderType='None', referencePriceType=0, account='U10394496', clearingIntent='IB', cashQty=0.0, dontUseAutoPriceForHedge=True, autoCancelDate='20240930 17:00:00 Central Standard Time', filledQuantity=0.0, refFuturesConId=2147483647, shareholder='Not an insider or substantial shareholder'),
 Order(permId=827631052, action='BUY', totalQuantity=1.0, orderType='LMT', lmtPrice=17261.0, auxPrice=0.0, tif='GTC', ocaType=3, orderRef='o96F4l68Y3GT', displaySize=2147483647, rule80A='0', openClose='', volatilityType=0, deltaNeutralOrderType='None', referencePriceType=0, account='U10394496', clearingIntent='IB', cashQty=0.0, dontUseAutoPriceForHedge=True, autoCancelDate='20240930 17:00:00 Central Standard Time', filledQuantity=0.0, refFuturesConId=2147483647, sharehol

In [6]:
ib.disconnect()